# IOTA + microgrid tests

In [1]:
from iota import ProposedTransaction, Address, Tag, TryteString, Iota, Transaction
from iota.crypto.types import Seed
import json
import pandapower as pp
import numpy as np
import time
from numpy.random import rand
import pandas as pd
import os
import pprint

### General parameters

In [2]:
def create_synthetic_data(d_steps, d_num_agents, d_t_gens):
    demand_df = pd.DataFrame(np.random.uniform(low=30, high=35, size=(d_num_agents, d_steps)))
    price_df = pd.DataFrame(np.random.uniform(low=12, high=15, size=(d_num_agents, d_steps)))
    supply_df = pd.DataFrame(np.random.uniform(low=66, high=70, size=(d_num_agents, d_steps)))
    gens = np.random.choice(demand_df.index, size = (d_t_gens,), replace=False)
    for gen in gens:
        price_df.iloc[gen] = np.zeros((1, d_steps))
        supply_df.iloc[gen] = np.zeros((1, d_steps))
    return demand_df, price_df, supply_df

### IOTA seed creation

In [3]:
demand = pd.read_csv(demand_file_path, index_col=0, header=None)
supply = pd.read_csv(supply_file_path, index_col=0, header=None)
price = pd.read_csv(price_file_path, index_col=0, header=None)
num_agents = demand.shape[0]
num_household = demand.shape[0]
steps = demand.shape[1]

In [4]:
supply
a_series = (supply != 0).any(axis=1)
new_df = supply.loc[a_series]

In [5]:
gen_nodes = np.array(new_df.index)

In [6]:
my_seed = Seed.random()
api = Iota('https://nodes.devnet.iota.org:443')
total_addresses = steps * (2 * num_agents + 1)
addresses = api.get_new_addresses(count=total_addresses)['addresses']

In [7]:
len(addresses)

1830

### Address generation

In [8]:
price_addresses_splits = []
money_addresses_splits = []
addresses1 = addresses[num_agents*steps:]
addresses2 = addresses[:num_agents*steps]
for agent in range(0, num_agents):
    price_addresses_splits.append(addresses1[agent*steps:agent*steps+steps])
    money_addresses_splits.append(addresses2[agent*steps:agent*steps+steps])
publish_addresses = addresses1[-steps:]

In [9]:
# Agent class definition

class Agent:
    def __init__(self, demand, supply, node, publish_address, price_address, money_address, price):
        """ Initialize agent instance.
        
        Args: 
            demand (list): list of floats with demand data
            supply (list): list of floats with supply. Ceros if no supply.
            node (float): number of the node.
            publish_address (list): list of iota.types.Address with the publish address.
            price_address (list): list of node's price addresses.
            money_address (list): list of node's money addresses.
            price (list): list of prices for supply offers.
        """
        self.demand = demand
        self.supply = supply
        self.node = node
        self.publish_address = publish_address
        self.price_address = price_address
        self.money_address = money_address
        self.consumption = demand-supply
        self.price = price
    
    def publish_info(self, data, address):
        """ Send transaction with data to specified address
        
        Args:
            data (dict): information to be sent.
            address (iota.types.Address): Destination address for information
        """
        # Build TX object
        tx = ProposedTransaction(
            address = Address(address),
            message = TryteString.from_unicode(json.dumps(data)),
            tag = Tag('INFO'),
            value = 0)
        # Send information
        tx = api.prepare_transfer(transfers=[tx])
        result = api.send_trytes(tx['trytes'], depth=3, min_weight_magnitude=9)
        return result
    
    def pay_power(self, step):
        """ Send power payment
        
        Args:
            step (float): step running at the moment.
        """
        unpack_data = self.check_address(step, 'price')
        for index, node in enumerate(unpack_data[0]['node']):
            data_to_send = {'payment': unpack_data[0]['price']*unpack_data[0]['power'][index]}
            address = self.address_dict[node]['money_address'][step]
            self.publish_info(data_to_send, address)
        
    def publish_energy_info(self, step):
        """ Build and send energy information
        
        Args:
            step (float): Step running at the moment.
        """
        data = {'node': self.node,
                'demand': self.demand[step],
                'supply': self.supply[step],
                'consumption': self.consumption[step],
                'price': self.price[step]}
        address = self.publish_address[step]
        self.publish_info(data, address)
        return data
    
    def check_address(self, step, address_type):
        """ Recover information from address.
        
        Args:
            step (float): Step running at the moment.
            address_type (str): Vector code to look for the address
        """
        
        if address_type == 'price':
            address = self.price_address[step]
        elif address_type == 'money':
            address = self.money_address[step]
        
        transactions = api.find_transactions(addresses=[address,])

        hashes = []
        for txhash in transactions['hashes']:
            hashes.append(txhash)

        trytes = api.get_trytes(hashes)['trytes']

        parts = []
        for trytestring in trytes:
            tx = Transaction.from_tryte_string(trytestring)
            parts.append((tx.current_index, tx.signature_message_fragment))

        parts.sort(key=lambda x: x[0])

        full_message = TryteString.from_unicode('')

        retrieved_data = []
        for index, part in parts:
            retrieved_data.append(json.loads(part.decode(errors='ignore')))
        return retrieved_data
    
    def get_demand(self):
        # Visualize demand
        return self.demand
    
    def get_supply(self):
        # Visualize supply
        return self.supply
    
    def get_node(self):
        # Visualize node
        return self.node
    
    def get_price_address(self):
        # Visualize price address
        return self.price_address
    
    def get_money_address(self):
        # Visualize money address
        return self.money_address
    
    def get_consumption(self):
        # Visualize consumption
        return self.consumption
    
    def get_prices(self):
        # Visualize price
        return self.price
    
    def assign_address_dict(self, address_dict):
        """ Assign address dict to node
        
        Args:
            address_dict (dict): With known addresses for each node (price and money) and publish addresses
        """
        self.address_dict = address_dict

### Agent creation

In [10]:
demand = pd.read_csv(demand_file_path, header=None, index_col=0)
supply = pd.read_csv(supply_file_path, header=None, index_col=0)
price = pd.read_csv(price_file_path, header=None, index_col=0)
num_agents = demand.shape[0]
num_household = demand.shape[0]
steps = demand.shape[1]

In [11]:
agents = []
for index in range(0, num_agents):
    agents.append(Agent(np.array(demand.iloc[index]), 
                       np.array(supply.iloc[index]), 
                       index, publish_addresses, 
                       price_addresses_splits[index], 
                       money_addresses_splits[index], 
                       np.array(price.iloc[index])))

In [12]:
def build_address_dictionary(agents, publish_addresses):
    agent_addresses = {}
    agent_addresses['publish_address'] = publish_addresses
    for index, agent in enumerate(agents):
        agent_addresses[index] = {'price_address': agent.get_price_address(),
                                  'money_address': agent.get_money_address()}
    return agent_addresses

In [13]:
def check_published_data(address):
    transactions = api.find_transactions(addresses=[address,])

    hashes = []
    for txhash in transactions['hashes']:
        hashes.append(txhash)
        
    trytes = api.get_trytes(hashes)['trytes']

    parts = []
    for trytestring in trytes:
        tx = Transaction.from_tryte_string(trytestring)
        parts.append((tx.current_index, tx.signature_message_fragment))

    parts.sort(key=lambda x: x[0])

    full_message = TryteString.from_unicode('')

    node_data = []
    for index, part in parts:
#         pprint.pprint(json.loads(part.decode(errors='ignore')))8.299+8.299+8.299+8.154
        node_data.append(json.loads(part.decode(errors='ignore')))

    return node_data

In [14]:
def single_sided_auction(node_data):
    node_vec = []
    supply_vec = []
    supply_price = []
    demand = []
    ED = 0
    for node in node_data:
        node_vec.append(node['node'])
        supply_vec.append(node['supply'])
        supply_price.append(node['price'])
        demand.append(node['demand'])
        ED += node['demand']
    supply_df = pd.DataFrame(data = {'Agents': node_vec, 'Supply': supply_vec, 'Price': supply_price, 'Demand': demand})
    supply_df = supply_df.sort_values(by='Price')
    cum_supply = np.cumsum(supply_df['Supply'])
    supply_df['Cumsum'] = cum_supply
    for index, value in enumerate(supply_df.Cumsum):
        if value >= ED:
            break
    return supply_df, ED, supply_df.iloc[index]['Price']

In [15]:
addresses_dict = build_address_dictionary(agents, publish_addresses)
for agent in agents:
    agent.assign_address_dict(addresses_dict)

### Microgrid creation

In [16]:
def micro_grid_exec(step):
    net = pp.create_empty_network()
    min_pu = 0.95
    max_pu = 1.05

    bus = []
    for index in range(len(agents)):
        bus.append(pp.create_bus(net, vn_kv=110, min_vm_pu=min_pu, max_vm_pu=max_pu))

    lines = []
    for index in range(len(agents)):
        if index < len(agents) - 1:
            lines.append(pp.create_line(net, bus[index], bus[index+1], length_km=1, std_type='149-AL1/24-ST1A 110.0'))
        else:
            lines.append(pp.create_line(net, bus[index], bus[0], length_km=1, std_type='149-AL1/24-ST1A 110.0'))


    for index in range(len(agents)):
        pp.create_load(net, bus[index], p_mw=supply_df['Demand'][index])

    gens = []
    for index in range(len(agents)):
        if supply.iloc[index][step] != 0:
            pp.create_gen(net, bus[index], p_mw=np.array(supply[step])[index], min_p_mw=0, max_p_mw=np.array(supply[step])[index], controllable=True, slack=True)
            
    pp.runpp(net)
    pf_result = net.res_gen
    
    return pf_result

In [17]:
nodes = list(np.linspace(0,num_agents-1, num_agents, dtype=np.int))

In [18]:
def payment_setup(step):
    power_per_agent = []
    for agent in agents:
        agent.get_node()
        demand = agent.demand[0]
        try:
            supply = gen_dict[agent.get_node()]
        except:
            supply = agent.get_supply()[0]
        to_app = supply - demand
        if abs(to_app) < 0.00001:
            to_app = 0
        power_per_agent.append(to_app)
    to_earn = {}
    to_pay = {}
    for index, payment in enumerate(power_per_agent):
        if payment < 0:
            to_pay[index] = abs(payment)
        else:
            to_earn[index] = payment
    
    payment_info = {}
    for payer, balance in to_pay.items():
        payment_info[payer] = {'node': [], 'total': []}
        for earner, gain in to_earn.items():
            if gain > 0:
                if balance >= gain:
                    payment_info[payer]['node'].append(earner)
                    payment_info[payer]['total'].append(gain)
                    to_pay[payer] = to_pay[payer] - gain
                    to_earn[earner] = to_earn[earner] - gain
                elif balance < gain:
                    payment_info[payer]['node'].append(earner)
                    payment_info[payer]['total'].append(balance)
                    to_pay[payer] = to_pay[payer] - balance
                    to_earn[earner] = to_earn[earner] - balance
                    break
    gather_data = {}
    for index, agent in enumerate(agents):
        info_dict = {}
        info_dict['price'] = price
        try:
            info_dict['node'] = payment_info[index]['node']
            info_dict['power'] = payment_info[index]['total']
        except:
            pass
        gather_data[index] = info_dict
#     print(power_per_agent)
    for index, agent in enumerate(agents):
        address = addresses_dict[index]['price_address'][step]
        data = gather_data[index]
        tx = ProposedTransaction(address=Address(address), message=TryteString.from_unicode(json.dumps(data)), tag=Tag('PRICE'),value=0)

        tx = api.prepare_transfer(transfers=[tx])

        result = api.send_trytes(tx['trytes'], depth=3, min_weight_magnitude=9)
    pay_nodes = set(nodes) - set(gen_nodes)
    for node in pay_nodes:
        agents[node].pay_power(step)

In [19]:
times_vec = []
for step in range(steps):
    start = time.time()
    for index, agent in enumerate(agents):
        agent.publish_energy_info(step)
    node_data = check_published_data(publish_addresses[step])
    supply_df, ED, price = single_sided_auction(node_data)
    pf_result = micro_grid_exec(step)
    supply_df.set_index('Agents', inplace=True)
    gen_values = pf_result['p_mw']
    gen_dict = dict(zip(gen_nodes, gen_values))
    total_demand = sum(supply_df['Demand'])
    total_supply = sum(gen_dict.values())
    losses = total_supply - total_demand
    payment_setup(step)
    print(price)
    times_vec.append(time.time() - start)

12.75134036
13.42212482
12.42212482
13.42212482
13.42212482
13.42212482
12.42212482
13.42212482
14.00176366
15.00176366
15.00176366
14.00176366
14.00176366
15.00176366
14.00176366
14.00176366
15.00176366
14.00176366
14.00176366
14.00176366
15.00176366
14.00176366
15.00176366
14.00176366
14.00176366
14.00176366
14.00176366
14.00176366
14.00176366
14.00176366


In [20]:
step = 0
for agent in agents:
    print(agent.check_address(step, 'price'))

[{'price': 12.75134036}]
[{'price': 12.75134036, 'node': [0, 3], 'power': [32.677331471138444, 33.3149762]}]
[{'price': 12.75134036, 'node': [3, 6, 7, 10, 13, 16, 18], 'power': [1.1428081534763734, 1.896156439999988, 2.0000000000000213, 33.17210010823512, 30.17146720341227, 2.2528049000000188, 33.67162466]}]
[{'price': 12.75134036}]
[{'price': 12.75134036, 'node': [18, 21], 'power': [0.09780507868064348, 1.7343729600000088]}]
[{'price': 12.75134036, 'node': [21], 'power': [2.000000000000007]}]
[{'price': 12.75134036}]
[{'price': 12.75134036}]
[{'price': 12.75134036, 'node': [21], 'power': [0.9999999999999787]}]
[{'price': 12.75134036, 'node': [21], 'power': [2.9999999999999787]}]
[{'price': 12.75134036}]
[{'price': 12.75134036, 'node': [21, 23, 24], 'power': [28.36862463126615, 2.0000000000000178, 31.41881976]}]
[{'price': 12.75134036, 'node': [24, 27, 28], 'power': [4.351501992601396, 34.103178909366775, 3.0000000000000213]}]
[{'price': 12.75134036}]
[{'price': 12.75134036, 'node': []

In [21]:
step = 0
print(agents[0].check_address(step, 'money'))
print(agents[3].check_address(step, 'money'))

[{'payment': 416.6797756450258}]
[{'payment': 424.8106006114994}, {'payment': 14.572335731160356}]


In [22]:
gen_dict

{0: 64.79235369168089,
 3: 67.12591656514849,
 4: 33.72123546,
 5: 34.72123546,
 6: 31.721235460000003,
 7: 32.72123546,
 8: 31.721235460000003,
 9: 34.72123546,
 10: 61.791338528746905,
 13: 66.12489426486388,
 14: 32.72123546,
 15: 31.721235460000003,
 16: 32.72123546,
 17: 34.72123546,
 18: 65.79254376125971,
 21: 65.12680953222696,
 22: 31.721235460000003,
 23: 32.72123546,
 24: 62.79218506470386,
 27: 66.12506645502287,
 28: 30.721235460000003,
 29: 34.72123546}

In [23]:
vec = []
for step in range(8):
    vec.append(agents[0].check_address(step, 'price')[0]['price'])

In [24]:
demand

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,,,,,,,,,,,,,,,,,,,,,
0,33.155090,34.228278,35.269981,32.265044,31.735561,30.920604,31.402899,32.721235,33.721235,32.721235,...,31.721235,30.721235,34.721235,34.721235,32.721235,34.721235,33.721235,34.721235,32.721235,32.721235
1,33.314976,34.853673,35.532571,32.301697,31.853297,30.822902,31.887549,32.396152,34.721235,33.721235,...,32.721235,31.721235,33.721235,30.721235,32.721235,32.721235,32.721235,34.721235,33.721235,34.721235
2,33.671625,34.854129,35.455646,32.357109,31.164639,30.705663,31.583547,32.608393,30.721235,32.721235,...,32.721235,33.721235,32.721235,34.721235,33.721235,32.721235,32.721235,31.721235,34.721235,30.721235
3,33.063164,34.631666,35.374026,32.949027,31.414728,30.178111,31.895472,32.910938,32.721235,32.721235,...,34.721235,33.721235,32.721235,33.721235,30.721235,31.721235,32.721235,34.721235,33.721235,31.721235
4,33.153193,34.445791,35.118582,32.028186,31.072765,30.244299,31.530618,32.660996,34.721235,32.721235,...,31.721235,34.721235,31.721235,31.721235,30.721235,33.721235,31.721235,30.721235,30.721235,34.721235
5,33.418820,34.485592,35.045856,32.742197,31.797849,30.331956,31.316986,32.541016,33.721235,31.721235,...,30.721235,31.721235,34.721235,32.721235,30.721235,32.721235,32.721235,34.721235,34.721235,31.721235
6,31.418820,34.418820,35.418820,31.418820,33.418820,35.418820,35.418820,31.418820,32.721235,33.721235,...,31.721235,30.721235,34.721235,32.721235,32.721235,31.721235,32.721235,32.721235,31.721235,32.721235
7,32.418820,31.418820,34.418820,35.418820,35.418820,34.418820,33.418820,33.418820,33.721235,34.721235,...,34.721235,31.721235,31.721235,31.721235,32.721235,30.721235,31.721235,30.721235,32.721235,33.721235
8,32.418820,35.418820,33.418820,33.418820,34.418820,34.418820,34.418820,32.418820,32.721235,32.721235,...,33.721235,30.721235,34.721235,33.721235,31.721235,31.721235,33.721235,31.721235,32.721235,31.721235


In [25]:
supply

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,,,,,,,,,,,,,,,,,,,,,
0,70.102725,71.871936,73.018188,68.512894,67.254255,65.647282,66.938322,69.037116,67.037116,70.037116,...,69.037116,66.037116,66.037116,67.037116,70.037116,67.037116,70.037116,69.037116,69.037116,67.037116
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,70.112946,72.487292,73.129137,68.709729,66.920336,65.998684,67.723010,68.698914,71.037116,67.037116,...,71.037116,69.037116,68.037116,66.037116,66.037116,66.037116,70.037116,67.037116,67.037116,71.037116
4,79.814858,71.990867,72.897817,68.836476,67.847909,65.315954,67.777872,69.124192,66.037116,68.037116,...,70.037116,67.037116,70.037116,71.037116,69.037116,69.037116,69.037116,70.037116,66.037116,67.037116
5,70.592717,72.312052,72.896946,68.616771,66.928618,66.757244,67.245876,68.874392,69.037116,66.037116,...,70.037116,71.037116,66.037116,71.037116,67.037116,71.037116,66.037116,68.037116,68.037116,67.037116
6,71.592717,71.592717,73.592717,72.592717,68.592717,68.592717,73.592717,71.592717,69.037116,71.037116,...,69.037116,71.037116,68.037116,66.037116,69.037116,66.037116,68.037116,66.037116,66.037116,71.037116
7,68.592717,70.592717,71.592717,70.592717,69.592717,73.592717,71.592717,69.592717,67.037116,71.037116,...,71.037116,68.037116,69.037116,69.037116,69.037116,69.037116,66.037116,69.037116,67.037116,66.037116
8,70.592717,71.592717,68.592717,73.592717,71.592717,72.592717,71.592717,71.592717,70.037116,68.037116,...,71.037116,67.037116,70.037116,70.037116,68.037116,68.037116,70.037116,71.037116,68.037116,66.037116


In [26]:
price

14.00176366

In [27]:
times_vec

[260.90573954582214,
 256.6680793762207,
 263.5863673686981,
 252.69940829277039,
 254.71967220306396,
 257.85832357406616,
 262.15488719940186,
 264.5509572029114,
 255.73674511909485,
 254.23591804504395,
 250.32464599609375,
 247.0682635307312,
 261.9412660598755,
 255.564115524292,
 253.64176511764526,
 252.91333532333374,
 258.44051480293274,
 249.78211402893066,
 248.78008699417114,
 253.85725378990173,
 250.0530664920807,
 260.8279483318329,
 262.6838674545288,
 255.58772468566895,
 254.97798323631287,
 256.19250297546387,
 260.6940801143646,
 257.7157154083252,
 259.58580470085144,
 271.57175636291504]